# bybit秒足作成プログラム.

- bybitの公開約定履歴から秒足を作成するプログラム.
- 分足や時間足の作成にも使えます.

In [1]:
import pandas as pd


## config.

In [2]:
from_date = "2021-01-01"
to_date = "2021-01-02"
interval = "5s"  # ex.1s, 5s, 10s, 1min, 1h


## データ取得と整形.

In [3]:
ohlcv = []
for d in pd.date_range(from_date, to_date):
    filename = f"https://public.bybit.com/trading/BTCUSD/BTCUSD{str(d)[0:10]}.csv.gz"
    # timestamp,symbol,side,size,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional
    # 1609545599.766732,BTCUSD,Sell,23099,29394.5,ZeroMinusTick,20bfb289-3707-5414-94ea-e996c85c62bf,78582728.0,23099,0.78582728
    # 1609545599.766732,BTCUSD,Sell,19379,29394.5,ZeroMinusTick,6bb10e16-dd85-5624-ae44-96088fab6b06,65927298.99999999,19379,0.65927299
    df = pd.read_csv(filename)[["timestamp", "price", "size"]]
    df = df.sort_values("timestamp")
    df = df[(df["timestamp"] >= d.timestamp()) & (df["timestamp"] < d.timestamp() + 60 * 60 * 24)]
    df.index = pd.to_datetime(df["timestamp"], unit="s", utc=True)
    df = pd.concat([
        df["price"].resample(interval).first(),
        df["price"].resample(interval).max(),
        df["price"].resample(interval).min(),
        df["price"].resample(interval).last(),
        df["size"].resample(interval).sum(),
    ], axis=1)
    df.columns = ["open", "high", "low", "close", "volume"]
    ohlcv.append(df)
    print(f"downloaded {filename}")

ohlcv = pd.concat(ohlcv)


downloaded https://public.bybit.com/trading/BTCUSD/BTCUSD2021-01-01.csv.gz
downloaded https://public.bybit.com/trading/BTCUSD/BTCUSD2021-01-02.csv.gz


In [4]:
ohlcv

,open,high,low,close,volume
timestamp,,,,,
2021-01-01 00:00:00+00:00,28955.0,28955.5,28955.0,28955.0,54827
2021-01-01 00:00:05+00:00,28955.0,28955.5,28955.0,28955.5,36391
2021-01-01 00:00:10+00:00,28955.0,28955.5,28955.0,28955.5,132512
2021-01-01 00:00:15+00:00,28955.0,28955.5,28955.0,28955.5,5971
2021-01-01 00:00:20+00:00,28955.5,28955.5,28955.0,28955.5,13602
...,...,...,...,...,...
2021-01-02 23:59:35+00:00,32209.0,32209.0,32208.5,32209.0,81226
2021-01-02 23:59:40+00:00,32208.5,32209.0,32208.5,32208.5,45297
2021-01-02 23:59:45+00:00,32209.0,32220.0,32208.5,32219.5,989692


## ファイル出力.

In [5]:
# ohlcv.to_csv(f"bybit_ohlcv_{interval}_{from_date.replace('-', '')}_{to_date.replace('-', '')}.csv")
